In [1]:
%pdb

Automatic pdb calling has been turned ON


In [1]:
%run dqn_atari.py --env "ALE/SpaceInvaders-v5" -o ./runs --seed 42

A.L.E: Arcade Learning Environment (version 0.10.1+unknown)
[Powered by Stella]
  5%|▍         | 50276/1050000 [01:35<1:00:03, 277.47it/s]

Step: 235/950000 Episode 95: Total reward: 90.0 Explore P: 0.9997885


  5%|▍         | 50756/1050000 [01:37<1:00:38, 274.66it/s]

Step: 716/950000 Episode 96: Total reward: 110.0 Explore P: 0.9993556


  5%|▍         | 51060/1050000 [01:38<55:17, 301.15it/s]  

...

Step: 125058/950000 Episode 325: Total reward: 110.0 Explore P: 0.8874478


 17%|█▋        | 175484/1050000 [08:46<51:03, 285.47it/s]

Step: 125447/950000 Episode 326: Total reward: 80.0 Explore P: 0.8870977


 17%|█▋        | 175599/1050000 [08:47<43:44, 333.20it/s]


KeyboardInterrupt: 